In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import re
import nltk
import spacy
nlp = spacy.load('en_core_web_sm')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stopwords = nlp.Defaults.stop_words
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
lemma=WordNetLemmatizer()
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from tensorflow.keras import layers
from keras.models import Sequential, Model
from keras.layers import Dense
from tensorflow.keras.layers import LSTM, Activation, Input, Dropout
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [3]:
df = pd.read_csv('/content/drive/MyDrive/NLP/train_sent.csv')
df.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [4]:
df.shape

(7920, 3)

In [5]:
df.isnull().sum()

id       0
label    0
tweet    0
dtype: int64

In [6]:
df['tweet'][45]

'Majestic We see it differently #photography #nature #landscape #forest #hidden #clearing #view #woods #tree #travel #biker #sony #a7 #ziess #photo #instgramhub #picoftheday https://www.instagram.com/p/BkvrsQJg-tN/?utm_source=ig_twitter_share&igshid=soka2bd9sb8h …'

In [7]:
def remove_urls(text):
  text = re.sub(r'https?://\S+|www\.\S+', ' ', text)
  return text

def remove_special_char(text):
  text = text.lower()
  text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
  return text

def remove_single_char(text):
  text = re.sub(r'\s[a-zA-Z]\s', ' ', text)
  return text

def remove_num(text):
  text = re.sub(r'\d+', ' ', text)
  return text

In [8]:
df['tweet'] = df['tweet'].apply(remove_urls)
df['tweet'] = df['tweet'].apply(remove_special_char)
df['tweet'] = df['tweet'].apply(remove_single_char)
df['tweet'] = df['tweet'].apply(remove_num)
df.head()

,id,label,tweet
0,1,0,fingerprint pregnancy test android apps ...
1,2,0,finally transparant silicon case thanks to ...
2,3,0,we love this would you go talk makememorie...
3,4,0,i wired know m george was made that way ip...
4,5,1,what amazing service apple won even talk to m...


In [9]:
def clean(text):
  tokens=word_tokenize(text)
  tokens_stop=[word for word in tokens if word not in stopwords]
  lemmatized_words=[lemma.lemmatize(token) for token in tokens if len(token)>2]
  clean_text=' '.join(lemmatized_words)
  return clean_text

df['tweet'] = df['tweet'].apply(clean)
df.head()

,id,label,tweet
0,1,0,fingerprint pregnancy test android apps beauti...
1,2,0,finally transparant silicon case thanks uncle ...
2,3,0,love this would you talk makememories unplug r...
3,4,0,wired know george wa made that way iphone cute...
4,5,1,what amazing service apple won even talk about...


In [10]:
df2 = pd.read_csv('/content/drive/MyDrive/NLP/test_sent.csv')
df2.head()

,id,tweet
0,7921,I hate the new #iphone upgrade. Won't let me d...
1,7922,currently shitting my fucking pants. #apple #i...
2,7923,"I'd like to puts some CD-ROMS on my iPad, is t..."
3,7924,My ipod is officially dead. I lost all my pict...
4,7925,Been fighting iTunes all night! I only want th...


In [11]:
df2['tweet'] = df2['tweet'].apply(remove_urls)
df2['tweet'] = df2['tweet'].apply(remove_special_char)
df2['tweet'] = df2['tweet'].apply(remove_single_char)
df2['tweet'] = df2['tweet'].apply(remove_num)
df2['tweet'] = df2['tweet'].apply(clean)
df2.head()

,id,tweet
0,7921,hate the new iphone upgrade won let download a...
1,7922,currently shitting fucking pant apple imac cas...
2,7923,like put some rom ipad that possible yes but w...
3,7924,ipod officially dead lost all picture and vide...
4,7925,been fighting itunes all night only want the m...


In [12]:
le = LabelEncoder()
y = le.fit_transform(df['label'])
y = y.reshape(-1,1)

In [33]:
x = df['tweet'].values
y = df['label'].values
test = df2['tweet'].values

In [34]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 100)

In [35]:
y_train = to_categorical(y_train,num_classes=2, dtype = 'float32')
y_test = to_categorical(y_test,num_classes=2, dtype = 'float32')

In [36]:
max_words = 1000
max_len = 100
tok = Tokenizer(num_words = max_words)
tok.fit_on_texts(x_train)
sequences = tok.texts_to_sequences(x_train)
sequences_matrix = sequence.pad_sequences(sequences, maxlen = max_len)
sequences_test = tok.texts_to_sequences(test)
sequences_matrix_test = sequence.pad_sequences(sequences_test, maxlen = max_len)

In [37]:
def RNN():
  inputs = Input(name='inputs',shape=[max_len])
  layer = Embedding(max_words,256,input_length=max_len)(inputs)
  layer = LSTM(512)(layer)
  layer = Dense(256,name='FC1')(layer)
  layer = Activation('relu')(layer)
  layer = Dropout(0.5)(layer)
  layer = Dense(2,name='out_layer')(layer)
  layer = Activation('softmax')(layer)
  model = Model(inputs=inputs,outputs=layer)
  return model

In [38]:
model = RNN()
model.summary()
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 100)]             0         
                                                                 
 embedding_4 (Embedding)     (None, 100, 256)          256000    
                                                                 
 lstm_4 (LSTM)               (None, 512)               1574912   
                                                                 
 FC1 (Dense)                 (None, 256)               131328    
                                                                 
 activation_8 (Activation)   (None, 256)               0         
                                                                 
 dropout_4 (Dropout)         (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 2)                 514 

In [39]:
model.fit(sequences_matrix,y_train,batch_size=128,epochs=10,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Epoch 1/10
47/47 [==============================] - 163s 3s/step - loss: 0.4368 - accuracy: 0.8067
Epoch 2/10
47/47 [==============================] - 167s 4s/step - loss: 0.2395 - accuracy: 0.9010
Epoch 3/10
47/47 [==============================] - 167s 4s/step - loss: 0.2049 - accuracy: 0.9168
Epoch 4/10
47/47 [==============================] - 164s 3s/step - loss: 0.1877 - accuracy: 0.9258
Epoch 5/10
47/47 [==============================] - 162s 3s/step - loss: 0.1678 - accuracy: 0.9348
Epoch 6/10
47/47 [==============================] - 162s 3s/step - loss: 0.1562 - accuracy: 0.9369
Epoch 7/10
47/47 [==============================] - 156s 3s/step - loss: 0.1409 - accuracy: 0.9471
Epoch 8/10
47/47 [==============================] - 158s 3s/step - loss: 0.1390 - accuracy: 0.9455
Epoch 9/10
47/47 [==============================] - 156s 3s/step - loss: 0.1196 - accuracy: 0.9508
Epoch 10/10
47/47 [==============================] - 157s 3s/step - loss: 0.1107 - accuracy: 0.9542


In [40]:
y_pred = model.predict(sequences_matrix_test)

In [41]:
y_pred = np.argmax(y_pred, axis=-1)
y_pred

array([1, 1, 1, ..., 1, 0, 0])

In [42]:
sample = pd.read_csv("/content/drive/My Drive/NLP/sample_sent.csv")

In [43]:
sample['label'] = y_pred

In [44]:
sample.to_csv('/content/drive/My Drive/NLP/submission_LSTM.csv', index=False)

In [45]:
sample.head()

,id,label
0,7921,1
1,7922,1
2,7923,1
3,7924,1
4,7925,1


In [46]:
sample['label'].value_counts()

0    1403
1     550
Name: label, dtype: int64